In [6]:
#@title Montar entorno local
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# Parse vars.env
with open('/content/drive/My Drive/Colab Notebooks/vars.env') as fp:
    lines = fp.readlines()

    for line in lines:
        key,var= line.strip().split('=')
        key = key.strip()
        var = var.strip()
        os.environ[key] = var

    # Change directory to local repository in Drive
    repo_path = os.getenv('SANDBOX_PATH')
    os.chdir(repo_path)



    !git status

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
On branch backend
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Backend/
	Interactive Backend.ipynb
	__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


#Login

In [4]:
from Backend.InteractiveSession import InteractiveSession
backend = InteractiveSession()

Box(children=(Text(value='', description='Username:'), Password(description='Password:'), Button(description='…

Logging in...
Successful login. Time until expire: 1 day, 0:00:00


#Tree


In [5]:
backend.backend.printTree()

0: MAPER   [GROUP]
    1: R. PEÑA 246   [GROUP]
        31: BOMBA 2   [MACHINE]
            82: A16   [MEASUREMENT_POINT]
                238: VMO AXIAL Z   [SENSOR]
                236: VMO HORIZONTAL X   [SENSOR]
                237: VMO VERTICAL Y   [SENSOR]
                235: TMO   [SENSOR]
            83: A03   [MEASUREMENT_POINT]
                244: VMI VERTICAL Y   [SENSOR]
                243: VMI HORIZONTAL X   [SENSOR]
                245: VMI AXIAL Z   [SENSOR]
                242: TMI   [SENSOR]
        33: BOMBA 1   [MACHINE]
            869: W1   [MEASUREMENT_POINT]
                905: VW1 - AXIAL - Z2   [SENSOR]
                906: VW1 - VERTICAL - X2   [SENSOR]
                873: VW1 - AXIAL - Z   [SENSOR]
                907: VW1 - HORIZONTAL - Y2   [SENSOR]
                872: VW1 - VERTICAL - X   [SENSOR]
                871: TW1   [SENSOR]
                870: VW1 - HORIZONTAL - Y   [SENSOR]
            80: A11   [MEASUREMENT_POINT]
                230: VMO 

#Menu

In [ ]:
# machines={
#             {50, "12/03/2020", "15/05/2020",cuando fallo, que falló},
#             {56, "05/05/2020", "15/06/2020",cuando fallo, que falló},
#             {45, "18/04/2020", "14/05/2020",cuando fallo, que falló}
#         }

# backend.iterate(machines,callback)


Acces token refreshed. Time until expire: 0:03:19.730749
Acces token refreshed. Time until expire: 0:03:19.009064


#De aca para abajo no mires, es codigo para eventualmente copiar y pegar

# Tests

In [ ]:
#@title Measurement selection

%load_ext google.colab.data_table
def getMeasurementsOnDataframe(FROM, TO):
  # Select the ID of the last dropwoen from previous menu.
  # IT IS NOT CHECKED IF ID BELONGS TO A SENSOR,  OTHERWISE THIS COULD FAIL !
  measurements = session.getMeasurements(dropdown_menu.children[-1].value,FROM,TO)

  df2 = pd.DataFrame(measurements['data'])
  del measurements['data']
  del measurements['type']
  del measurements['node_id']
  df1 = pd.DataFrame(measurements)
  df = pd.concat([df1, df2], axis=1, sort=False)
  df['dates'] = df['dates'].transform(lambda x: datetime.fromtimestamp(int(x/1000)))
  df.set_index('measurements_ids',inplace=True)

  display(df)

  # df2 = pd.DataFrame(measurements['data'])
  # del measurements['data']
  # del measurements['type']
  # del measurements['node_id']
  # df1 = pd.DataFrame(measurements)
  # df = pd.concat([df1, df2], axis=1, sort=False)
  # df['dates'] = df['dates'].transform(lambda x: datetime.fromtimestamp(int(x/1000)))
  # df.set_index('measurements_ids',inplace=True)

  # display(df[df.a_rms!=0].sort_values('dates'))

interact_manual(getMeasurementsOnDataframe,
        FROM=widgets.DatePicker(value=date.today()-timedelta(days=5)),
        TO=widgets.DatePicker(value=date.today()))


Box(children=(Text(value='admin', description='Username:'), Password(description='Password:'), Button(descript…

VBox()

NameError: ignored

In [ ]:
# measID = 6054883

# printMeasurement(session.getMeasurement (measID))

# measurement,domain,freq,N = session.getMeasurementCooked(measID)


# # Rename some variables
# if domain=='time':
#   signal = measurement
#   fs = freq
#   nsamples = N
#   time = np.linspace(0, N/fs, N)
#   plt.plot(time,signal)
#   plt.xlabel('Time [s]')
# else:
#   spectrum = measurement
#   fmax = freq
#   nlines = N
#   freq = np.linspace(0, fmax, N+1)
#   deltaf = nlines/fmax
#   plt.plot(freq,spectrum)
#   plt.xscale('log')
#   plt.xlabel('Frequency [Hz]')

# meas = session.getMeasurement(26486);
# printMeasurement(meas)

# domain = meas['info']['domain']

# if domain=='time':
#   signal = np.frombuffer(base64.b64decode(meas['raw_data'][domain]['acceleration']), dtype=np.float32)
#   fs = meas['info']['measurement_parameters']['fs']
#   N = meas['info']['measurement_parameters']['samples']
#   time = np.linspace(0, N/fs, N)
#   plt.plot(time,signal)
# else:
#   spectrum = np.frombuffer(base64.b64decode(meas['raw_data'][domain]['acceleration']), dtype=np.float32)
#   fmax = meas['info']['measurement_parameters']['fmax']
#   N = meas['info']['measurement_parameters']['lines']

#   freq = np.linspace(0, fmax, N+1)
#   plt.plot(freq,spectrum)
#   plt.xscale('log')

# Batch test

In [ ]:
def F0Detect(freq,spectrum):
#######################################
    peak_indexs, peak_properties = find_peaks(spectrum,prominence=(None,None))

    mu = np.mean(peak_properties['prominences'])
    std = np.std(peak_properties['prominences'])

    peak_indexs = [peak_indexs[i] for i in range(len(peak_indexs)) if peak_properties['prominences'][i]> mu+0.25*std]
##########################################

    f0, f0error = TWM_p(freq[peak_indexs], spectrum[peak_indexs], np.linspace(22.5,27.5,100))
    #f0, f0error = TWM_p(pfreq, pmag, f0cf)
    #plt.plot(f0,f0error,lw=0.8)
    f0 = f0[np.argmin(f0error)]
    return f0
#######################

  # else:
  #   raise()

In [ ]:
def getMeasurementInTimeRange(id,time_begin,time_stop):

  measurements = session.getMeasurements(id,time_begin,time_stop)
  #print(measurements['data']['a_rms'])

  if measurements is not None and len(measurements['measurements_ids'])>0:
    i = next((i for i, a_rms in enumerate(measurements['data']['a_rms']) if a_rms!=None and a_rms>0), None)
    if i is not None:
      return session.getMeasurement(measurements['measurements_ids'][i])
    else:
      return None
  else:
    return None

# Receives a measurement point and a time range

def processMeasurementPoint(node,date_from,date_to,rpm):

  assert node['type']=='MEASUREMENT_POINT'

  machine_id = node.get('parent_id')
  machine_name = session.getNode(machine_id).get('name')

  #d = kwargs['date_from']
  date_from = datetime(date_from.year, date_from.month, date_from.day)

  #d = kwargs['date_to']
  date_to = datetime(date_to.year, date_to.month, date_to.day) 

  date_delta = timedelta(hours=6)

  # Remove temperature 
  axes = [axis for axis in node['children'] if not re.match('T\w{2}',axis['name'])]

  f0history = []
  f0dates = []
  rms_values = []
  rms_dates = []

  # Iterate over range of dates
  total_seconds = (date_to-date_from).total_seconds()

  curr_time = date_from
  while curr_time <= date_to:
    # Print progress.
    progress = (curr_time-date_from).total_seconds() / total_seconds *100
    print(f"{machine_name} - {node['name']} -> {progress}%")

    # Get a measurement for each axis in a time range
    axis_meas = []
    axis_rms = []
    for n,axis in enumerate(axes):
      meas = getMeasurementInTimeRange(axis['id'], curr_time-date_delta/2, curr_time+date_delta/2)
      axis_meas.append(meas)
      axis_rms.append(meas['data']['a_rms'] if meas is not None else None)


    # If all axis have measurement
    if all(m is not None for m in axis_meas):

      rms_values.append(axis_rms)    
      rms_dates.append(datetime.fromtimestamp(meas['date']/1000))

      f0axis = []
      for i in range(len(axis_meas)):
        meas = axis_meas[i]

        assert meas['info']['domain']=='frequency'

        spectrum = np.frombuffer(base64.b64decode(meas['raw_data']['frequency']['acceleration']), dtype=np.float32)
        fmax = meas['info']['measurement_parameters']['fmax']
        N = meas['info']['measurement_parameters']['lines']
        freq = np.linspace(0, fmax, N+1)

        # Create big figure 
        fig = plt.figure(figsize=(18,16), constrained_layout=False)

        # Run test
        test_result = fundamental_detect_test(freq,spectrum,f0nom=rpm/60,fig=fig)

        f0axis.append(test_result)

        name = f"{node['name']} - {axis['name']} - {datetime.fromtimestamp(meas['date']//1000)}"
        plt.suptitle(name, fontsize=25)
        plt.tight_layout()
        plt.subplots_adjust(top=0.92)
        plt.savefig(os.getcwd() + '/' + name + ".png")
        plt.close()


        # #######################################
        # peak_indexs, peak_properties = find_peaks(spectrum,prominence=(None,None))

        # mu = np.mean(peak_properties['prominences'])
        # std = np.std(peak_properties['prominences'])

        # peak_indexs = [peak_indexs[i] for i in range(len(peak_indexs)) if peak_properties['prominences'][i]> mu+0.25*std]
        # ##########################################

        # f0, f0error = TWM_p(freq[peak_indexs], spectrum[peak_indexs], np.linspace(22.5,27.5,100))
        # #f0, f0error = TWM_p(pfreq, pmag, f0cf)
        # #plt.plot(f0,f0error,lw=0.8)
        # f0 = f0[np.argmin(f0error)]
        # #######################

        # plt.plot(freq,spectrum)
        # plt.plot(freq[peak_indexs], spectrum[peak_indexs], 'rx')
        # plt.vlines(x=np.arange(start=f0,step=f0,stop=fmax),ymin=min(spectrum),ymax=max(spectrum),ls=':',lw=0.8)
        # plt.legend([f"{axis['name']} - f0={f0:.4f}hz"])
    
      f0dates.append(datetime.fromtimestamp(meas['date']/1000))
      f0history.append(f0axis)

    else:
      print(f"X {axis_meas[0]==None} Y {axis_meas[1]==None} Z {axis_meas[2]==None} ")
      pass

    curr_time += date_delta

  # Restore enline character
  #print(" ", end = '\n')

  # 
  # |date|axis|algorithm| -> |axis|date|algorithm|
  f0history = np.array(f0history)
  print(f0history.shape)
  f0history = np.transpose(f0history, [1,0,2])
  print(f0dates)
  f0dates = matplotlib.dates.date2num(f0dates)

  #############################################################################
  # Create big figure 
  fig = plt.figure(figsize=(18,16), constrained_layout=False)

  fig.add_subplot(3,1,1)
  plt.plot_date(f0dates, f0history[0], ls='-')
  plt.title('X')
  plt.legend(['NAIVE', 'TWMM', 'COMB'])

  fig.add_subplot(3,1,2)
  plt.plot_date(f0dates, f0history[1], ls='-')
  plt.title('Y')
  plt.legend(['NAIVE', 'TWMM', 'COMB'])

  fig.add_subplot(3,1,3)
  plt.plot_date(f0dates, f0history[2], ls='-')
  plt.title('Z')
  plt.legend(['NAIVE', 'TWMM', 'COMB'])

  name = f"{node['name']} - {date_from.date()} - {date_to.date()} Rotating Frequency"

  plt.suptitle(name,fontsize=25)
  plt.tight_layout()
  plt.subplots_adjust(top=0.92)
  plt.savefig(os.getcwd() + '/' + name + ".png")
  plt.close()
  #############################################################################
  # Create big figure 
  fig = plt.figure(figsize=(18,16), constrained_layout=False)
  rms_dates = matplotlib.dates.date2num(rms_dates)
  plt.plot_date(rms_dates, rms_values, ls='-')
  plt.legend(['X', 'Y', 'Z'])
  name = f"{node['name']} - {date_from.date()} - {date_to.date()} RMS"
  plt.title(name,fontsize=25)
  plt.tight_layout()
  plt.savefig(os.getcwd() + '/' + name + ".png")
  plt.close()
  #############################################################################
  print(f"FINISH - Measurements processed: {f0dates.shape[0]}")



In [ ]:
def print_pretty(d, indent=0):
   for key, value in d.items():
      if isinstance(value, dict):
         print('\t' * indent + str(key))
         print_pretty(value, indent+1)
      else:
         print('\t' * indent + str(key) + ": " + str(value))

def processSelection(FROM, TO,HS_STEP):

  if len(dropdown_menu.children)==0:
    print("Please login bebore running processing")
    return


  ##############################################################################
  dropdown_ids = [child.value for child in dropdown_menu.children]
  if dropdown_ids[-1] is not None:
    # Sensor axis selected
    ID = dropdown_ids[-1]
  elif len(dropdown_menu.children)>1:
    ID = dropdown_ids[-2]
  else:
    #Nothing selected
    return 
  ##############################################################################
  # Create base dir from dropdown menu names
  if dropdown_ids[-1] is None:
    dropdown_names = [child.options[child.index][0].replace(':',' -') for child in dropdown_menu.children[0:-1]]
  else:
    dropdown_names = [child.options[child.index][0].replace(':',' -') for child in dropdown_menu.children]

  base_path = './'+'/'.join(dropdown_names)

  os.chdir('/content/')
  cwd_backup = os.getcwd()

  print(f"Stored current directory: {cwd_backup}")

  if not os.path.exists(base_path):
    os.makedirs(base_path)
  os.chdir(base_path)
  ##############################################################################
  # 
  # Print tree
  #session.printTree(ID)

  # Get subtree
  node = getSubtree(session.tree,ID)
  print(node.get('name'))
################################################################################
  if node['type']=='MEASUREMENT_POINT':
    machine =  session.getNode(node.get('parent_id'))
    machineInfo = machine['static_data']['description']['mechanical']
    drivingStage = machineInfo.get('driving_stage')
################################################################################
  # Initialize traverse stack
  stack = [node,]
  paths = ['.', ]
  indents = [0,]

  # Traverse tree
  while stack: 
    node = stack.pop()
    path = paths.pop()
################################################################################
    if node['type']=='GROUP':
      print(f"Group: {node['name']}")
      group = node
################################################################################
    if node['type']=='MACHINE':
      print(f"Machine: {node['name']}")
      machine = node
      machineInfo = node['static_data']['description']['mechanical']
      drivingStage = machineInfo.get('driving_stage')
      
      # print()
      # print(path)
      # print_pretty(machineInfo)
      # print()
################################################################################
    if node['type']=='MEASUREMENT_POINT':
      # Process sensors installed in the motor with information loaded 
      if drivingStage is not None:
        print(f"Measurement point: {node['name']}")
        ts = drivingStage.get('ts')
      else:
        print("No information on driving stage!")
        ts = 1500

        # Only process sensors installed in the motor
      #if "MI" in node['name'] or "MO" in node['name']:
      print(f"Ts: {ts}")
      # Create folder for sensor reports
      if not os.path.exists(path):
        os.makedirs(path)

      os.chdir(path)
      print(os.getcwd())
      processMeasurementPoint(node,FROM,TO,rpm=ts)

      os.chdir('/content/' + base_path)
       
################################################################################
    if node['type']=='SENSOR':
      pass
################################################################################

    # Uncomment to print while iterating:
    #
    # colors = {'GROUP': 'red',
    #   'MACHINE': 'green',
    #   'MEASUREMENT_POINT': 'magenta',
    #   'SENSOR': 'blue'}
    # c = colors[node['type']]
    # node_id = colored(node['id'],c)
    # node_name = colored(node['name'], c, attrs=['bold'])
    # node_type = colored(node['type'],c)
    # indent = indents.pop()
    # print(' '*indent + node_id + ': '+ node_name + '   ['+ node_type + ']')
    # indents.extend([indent+2 for i in range(len(node['children']))])

    # Add current node children to stack
    stack.extend(reversed(node['children'])) 
    paths.extend([f"{path}/{child['id']} - {child['name']}" for child in reversed(node['children'])])

  # Restore backed up path
  os.chdir(cwd_backup)

In [ ]:
# from google.colab import drive
# import os

# drive.mount('/content/drive')

# notebook_dir = '/content/drive/My Drive/Maper Algorithm Sandbox'

# if not os.path.exists(notebook_dir):
#   os.mkdir(notebook_dir)




In [ ]:
display(login_menu)

display(dropdown_menu)

interact_manual(processSelection,
        FROM=widgets.DatePicker(value=date.today()-timedelta(days=60)),
        TO=widgets.DatePicker(value=date.today()-timedelta(days=20)),
        HS_STEP = widgets.IntSlider(value=6,min=1,max=24))
print()

Box(children=(Text(value='admin', description='Username:'), Password(description='Password:'), Button(descript…

interactive(children=(DatePicker(value=datetime.date(2020, 4, 2), description='FROM'), DatePicker(value=dateti…

In [ ]:
#!zip -r F0_DETECTION_IFF "/content/2 - IFF"
!zip -r F0_DETECTION "/content/2 - IFF" "/content/18 - ESTOPAPLAST sa"
#from colab import files
#files.download("F0_DETECTION.zip")

	zip warning: name not matched: /content/2 - IFF
	zip warning: name not matched: /content/18 - ESTOPAPLAST sa

zip error: Nothing to do! (try: zip -r F0_DETECTION . -i /content/2 - IFF /content/18 - ESTOPAPLAST sa)


In [ ]:
#!pip install objgraph
import objgraph
objgraph.show_most_common_types(limit=100)


ModuleNotFoundError: ignored

In [ ]:
# drive.flush_and_unmount()
# print('All changes made in this colab session should now be visible in Drive.')

In [ ]:
print(os.getcwd())